In [1]:
cd ../../../Datasets/190829_Kanshan_zjfx

F:\Codes\Datasets\190829_Kanshan_zjfx


In [2]:
ls

 驱动器 F 中的卷是 存储
 卷的序列号是 C14D-581B

 F:\Codes\Datasets\190829_Kanshan_zjfx 的目录

2019/10/21  18:51    <DIR>          .
2019/10/21  18:51    <DIR>          ..
2019/09/26  19:08     5,805,965,112 answer_info_0926.txt
2019/09/26  18:10       333,836,065 invite_info_0926.txt
2019/09/28  12:14        37,860,903 invite_info_evaluate_1_0926.txt
2019/09/26  17:12       552,574,982 member_info_0926.txt
2019/09/26  18:01     1,074,273,891 question_info_0926.txt
2019/10/18  16:35    <DIR>          sample
2019/08/05  17:19        15,951,125 single_word_vectors_64d.txt
2019/08/05  17:20        69,411,196 topic_vectors_64d.txt
2019/08/05  17:19     1,230,810,962 word_vectors_64d.txt
               8 个文件  9,120,684,236 字节
               3 个目录 421,294,071,808 可用字节


## 0.1 超参数

In [13]:
import torch

BATCH_SIZE = 8192
DO_DATASET_BALANCE = False
EPOCH = 32
LR = 0.01
TRAIN_ALL = False

PKL_DIR_READ = '../../Jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
PKL_DIR_OUT = '../../Jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

## 0.2 数据分析

In [3]:
import pandas as pd
user_info = pd.read_csv('member_info_0926.txt', header=None, sep='\t')
user_info.columns = ['用户id','性别','创作关键词','创作数量等级','创作热度等级','注册类型','注册平台','访问评率','用户二分类特征a','用户二分类特征b','用户二分类特征c','用户二分类特征d','用户二分类特征e','用户多分类特征a','用户多分类特征b','用户多分类特征c','用户多分类特征d','用户多分类特征e','盐值','关注话题','感兴趣话题']
print('每列类别数：')
for col in user_info.columns:
    print(col, len(user_info[col].unique()))

每列类别数：
用户id 1931654
性别 3
创作关键词 1
创作数量等级 1
创作热度等级 1
注册类型 1
注册平台 1
访问评率 5
用户二分类特征a 2
用户二分类特征b 2
用户二分类特征c 2
用户二分类特征d 2
用户二分类特征e 2
用户多分类特征a 2561
用户多分类特征b 291
用户多分类特征c 428
用户多分类特征d 1556
用户多分类特征e 2
盐值 732
关注话题 1301436
感兴趣话题 1399721


In [4]:
question_info = pd.read_csv('question_info_0926.txt', header=None, sep='\t')
question_info.columns = ['问题id','问题创建时间','问题标题单字编码','问题标题切词编码','问题描述单字编码','问题描述切词编码','问题绑定话题']
print('每列类别数：')
for col in question_info.columns:
    print(col, len(question_info[col].unique()))

每列类别数：
问题id 1829900
问题创建时间 54617
问题标题单字编码 1828611
问题标题切词编码 1786981
问题描述单字编码 831554
问题描述切词编码 816606
问题绑定话题 1166444


## 1.1 加载数据集

In [4]:
import pandas as pd

# 导入数据
user_info = pd.read_csv('member_info_0926.txt', header=None, sep='\t')
question_info = pd.read_csv('question_info_0926.txt', header=None, sep='\t')
train = pd.read_csv('invite_info_0926.txt', header=None, sep='\t')
test = pd.read_csv('invite_info_evaluate_1_0926.txt', header=None, sep='\t')

user_info.columns = ['用户id','性别','创作关键词','创作数量等级','创作热度等级',
                     '注册类型','注册平台','访问评率','用户二分类特征a',
                     '用户二分类特征b','用户二分类特征c','用户二分类特征d',
                     '用户二分类特征e','用户多分类特征a','用户多分类特征b',
                     '用户多分类特征c','用户多分类特征d','用户多分类特征e',
                     '盐值','关注话题','感兴趣话题']
user_info  = user_info.drop(['创作关键词','创作数量等级','创作热度等级',
                             '注册类型','注册平台'], axis=1)
question_info.columns = ['问题id','问题创建时间','问题标题单字编码',
                         '问题标题切词编码','问题描述单字编码',
                         '问题描述切词编码','问题绑定话题']

train.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
train = pd.merge(train, user_info, how='left', on='用户id')
train = pd.merge(train, question_info, how='left', on='问题id')

test.columns = ['问题id', '用户id', '邀请创建时间']
test = pd.merge(test, user_info, how='left', on='用户id')
test = pd.merge(test, question_info, how='left', on='问题id')

print('ok')

ok


In [11]:
# 数据合并
data = pd.concat([train, test], axis=0, sort=True)

# 用于保存提交结果
result_append = data[['问题id', '用户id', '邀请创建时间']][train.shape[0]:]

# 数据处理
data['邀请创建时间-day'] = data['邀请创建时间'].apply(lambda x:x.split('-')[0].split('D')[1])
data['邀请创建时间-hour'] = data['邀请创建时间'].apply(lambda x:x.split('-')[1].split('H')[1])
data['问题创建时间-day'] = data['问题创建时间'].apply(lambda x:x.split('-')[0].split('D')[1])
data['问题创建时间-hour'] = data['问题创建时间'].apply(lambda x:x.split('-')[1].split('H')[1])

data['邀请问题间隔-day'] = data['邀请创建时间-day'].astype('int') - data['问题创建时间-day'].astype('int')

data['关注话题命中数'] = data['关注话题'] + ',' + data['问题绑定话题']
data['关注话题命中数'] = data['关注话题命中数'].apply(lambda x: len(x.split(',')) - len(set(x.split(','))))

# 删除的特征并非不重要，相反这部分的数据很重要，如何处理这部分特征有很大的发挥空间，本baseline不涉及这些特征。
drop_feat = ['问题标题单字编码','问题标题切词编码','问题描述单字编码',
             '问题描述切词编码','问题绑定话题', '关注话题','感兴趣话题',
             '问题创建时间','邀请创建时间-day','邀请创建时间-hour',
             '问题创建时间-day','问题创建时间-hour'#,'邀请创建时间'
            ]
data  = data.drop(drop_feat, axis=1)
data = data[['问题id', '用户id', '邀请创建时间', '性别', '是否回答',
             '用户二分类特征a', '用户二分类特征b', '用户二分类特征c',
             '用户二分类特征d', '用户二分类特征e', '用户多分类特征a',
             '用户多分类特征b', '用户多分类特征c', '用户多分类特征d',
             '用户多分类特征e', '盐值', '访问评率', '邀请问题间隔-day',
             '关注话题命中数']]

print(data.columns)

Index(['问题id', '用户id', '邀请创建时间', '性别', '是否回答', '用户二分类特征a', '用户二分类特征b',
       '用户二分类特征c', '用户二分类特征d', '用户二分类特征e', '用户多分类特征a', '用户多分类特征b', '用户多分类特征c',
       '用户多分类特征d', '用户多分类特征e', '盐值', '访问评率', '邀请问题间隔-day', '关注话题命中数'],
      dtype='object')


## 1.2 处理数据集

离散特征的`LabelEncoder`编码。

In [12]:
from sklearn.preprocessing import LabelEncoder

# 离散值编码
class_feat = [#'用户id','问题id',
              '性别', '访问评率','用户多分类特征a','用户多分类特征b',
              '用户多分类特征c','用户多分类特征d','用户多分类特征e']
encoder = LabelEncoder()
for i in class_feat:
    data[i] = encoder.fit_transform(data[i])
    
# 连续值标准化
class_feat = ['用户二分类特征a', '用户二分类特征b', '用户二分类特征c', 
              '用户二分类特征d', '用户二分类特征e', '用户多分类特征e',
              '盐值', '访问评率', '邀请问题间隔-day', '关注话题命中数']
for i in class_feat:
    data[i] = (data[i] - data[i].mean(axis=0)) / data[i].std(axis=0)
    
data.iloc[:10]

,问题id,用户id,邀请创建时间,性别,是否回答,用户二分类特征a,用户二分类特征b,用户二分类特征c,用户二分类特征d,用户二分类特征e,用户多分类特征a,用户多分类特征b,用户多分类特征c,用户多分类特征d,用户多分类特征e,盐值,访问评率,邀请问题间隔-day,关注话题命中数
0,Q2166419046,M401693808,D3865-H22,2,0.0,-1.404297,1.532980,-0.201699,-0.591583,-0.283857,1917,170,244,799,0.343009,-1.133318,1.161576,-0.147008,1.313411
1,Q1550017551,M3392373099,D3844-H11,2,0.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,1092,115,249,657,-2.915374,-1.036384,-0.456167,-0.035249,-0.438016
2,Q604029601,M2317670257,D3862-H15,2,0.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,1380,206,0,440,0.343009,-0.093474,1.161576,-0.166730,-0.438016
3,Q2350061229,M1618461867,D3849-H11,2,0.0,0.712100,-0.652324,-0.201699,1.690380,-0.283857,1092,115,244,799,0.343009,-1.062820,-0.995415,-0.160156,-0.438016
4,Q2443223942,M3544409350,D3867-H4,2,0.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,506,204,175,705,0.343009,-1.142130,-0.456167,-0.041823,-0.438016
5,Q640765464,M2818659842,D3841-H16,1,0.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,1380,206,0,440,0.343009,-0.401902,-0.995415,-0.133860,1.313411
6,Q795459266,M2818659842,D3861-H20,1,0.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,1380,206,0,440,0.343009,-0.401902,-0.995415,1.430767,-0.438016
7,Q190554387,M1581217469,D3850-H8,2,1.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,1092,115,0,128,0.343009,0.831811,-0.456167,-0.173304,-0.438016
8,Q1958712851,M3021021791,D3850-H19,2,0.0,-1.404297,1.532980,-0.201699,-0.591583,-0.283857,1486,170,249,657,0.343009,-1.265502,1.161576,-0.166730,-0.438016
9,Q311993584,M1766315480,D3839-H15,1,0.0,0.712100,-0.652324,-0.201699,-0.591583,-0.283857,1092,115,370,1252,0.343009,0.179706,-0.995415,-0.160156,-0.438016


## 1.3 包装数据集

In [14]:
import numpy as np

if DO_DATASET_BALANCE:
    temp = data[data['是否回答'] == 1]  # 用于均衡的数据
    train_x = data[~np.isnan(data['是否回答'])].append([temp, temp, temp, temp]).values
else:
    train_x = data[~np.isnan(data['是否回答'])].values
train_x = train_x[['性别', '是否回答',
             '用户二分类特征a', '用户二分类特征b', '用户二分类特征c',
             '用户二分类特征d', '用户二分类特征e', '用户多分类特征a',
             '用户多分类特征b', '用户多分类特征c', '用户多分类特征d',
             '用户多分类特征e', '盐值', '访问评率', '邀请问题间隔-day',
             '关注话题命中数']]
train_y = train_x[:, 4]

test_x = data[np.isnan(data['是否回答'])].values
test_x = test_x[['性别', '是否回答',
             '用户二分类特征a', '用户二分类特征b', '用户二分类特征c',
             '用户二分类特征d', '用户二分类特征e', '用户多分类特征a',
             '用户多分类特征b', '用户多分类特征c', '用户多分类特征d',
             '用户多分类特征e', '盐值', '访问评率', '邀请问题间隔-day',
             '关注话题命中数']]
test_y = test_x[:, 4]
print('ok.')

ok.


In [15]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, mode='train'):
        self.mode = mode
        if mode == 'train':
            self.data = {
                'x': train_x,
                'y': train_y,
            }
        elif mode == 'test':
            self.data = {
                'x': test_x,
                'y': test_y,
            }
        else:
            raise
            
    def __getitem__(self, index):
        return self.data['x'][index], self.data['y'][index]
        
    def __len__(self):
        return len(self.data['x'])
    
    
print('ok.')

ok.


In [16]:
from torch.utils.data import random_split

full_train_dataset = MyDataset(mode='train')
test_dataset = MyDataset(mode='test')
train_size, val_size, test_size = 0, 0, 0

if TRAIN_ALL:
    data_loader = {
        'train': DataLoader(dataset=full_train_dataset, 
                            batch_size=BATCH_SIZE, 
                            shuffle=True),
        'test': DataLoader(dataset=test_dataset,
#                            batch_size=BATCH_SIZE,
                           shuffle=False),
    }
    train_size = len(full_train_dataset)
    test_size = len(test_dataset)
else:
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

    data_loader = {
        'train': DataLoader(dataset=train_dataset, 
                            batch_size=BATCH_SIZE, 
                            shuffle=True),
        'val': DataLoader(dataset=val_dataset, 
                          batch_size=BATCH_SIZE,
                          shuffle=False),
        'test': DataLoader(dataset=test_dataset,
#                            batch_size=BATCH_SIZE,
                           shuffle=False),
    }
    train_size = len(train_dataset)
    val_size = len(val_dataset)
    test_size = len(test_dataset)
    
print('train_size:', len(train_dataset))
print('val_size:', len(val_dataset))
print('test_size:', len(test_dataset))

train_size: 7591329
val_size: 1897833
test_size: 1141683


## 2.1 构建网络

In [10]:
from my_model import MyNet

net = MyNet().to(DEVICE)
net

MyNet(
  (embed_sex): Embedding(3, 2)
  (embed_multi_a): Embedding(2561, 13)
  (embed_multi_b): Embedding(291, 6)
  (embed_multi_c): Embedding(428, 7)
  (embed_multi_d): Embedding(1556, 11)
  (fc1): Linear(in_features=49, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=16, bias=True)
  (fc7): Linear(in_features=16, out_features=2, bias=True)
  (relu): ReLU(inplace)
)

## 2.2 优化器、损失函数

In [67]:
from torch import optim, nn

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.75)

'ok'

'ok'

## 2.3 训练

In [18]:
import math

# 计算一轮enumerate的长度
train_len = math.ceil(train_size / BATCH_SIZE)
val_len = math.ceil(val_size / BATCH_SIZE)
test_len = math.ceil(test_size / BATCH_SIZE)

print('train_len:', train_len)
print('val_len:', val_len)
print('test_len:', test_len)

train_len: 1585
val_len: 397
test_len: 140


In [68]:
from tqdm import tqdm

for epoch in range(EPOCH):
    count, running_correct, running_loss = 0, 0, 0
    
    net.train()
    with tqdm(total=train_len) as pbar:
        for step, (bx, by) in enumerate(data_loader['train']):
            # 训练
            bx = bx.float().to(DEVICE)
            by = by.long().to(DEVICE)

            prediction = net(bx)
            loss = loss_function(prediction, by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 统计成效
            pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
            my_loss = float(loss.data.cpu().numpy())
            my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
            my_acc = my_correct / BATCH_SIZE
            count += 1
            running_loss += my_loss
            running_correct += my_correct
            
            # 更新进度条
            pbar.update(1)
            pbar.set_description('Epoch: %d, Loss: %.4f, Acc: %.4f' % (
                                  epoch, 
                                  my_loss, 
                                  my_acc))
    
    # 输出一轮结果
    print('Epoch: %d, Loss: %.4f, Acc: %.4f, Lr: %.5f' % (
           epoch,
           running_loss / count,
           running_correct / train_size,
           optimizer.state_dict()['param_groups'][0]['lr']))
    
    # 保存参数
    torch.save(net.state_dict(), PKL_DIR_OUT)
    
    # lr scheduler
    scheduler.step()

Epoch: 0, Loss: 0.6504, Acc: 0.0389: 100%|██████████| 1585/1585 [06:53<00:00,  3.83it/s]


Epoch: 0, Loss: 0.6321, Acc: 0.6247, Lr: 0.00010


Epoch: 1, Loss: 0.6156, Acc: 0.0436: 100%|██████████| 1585/1585 [08:07<00:00,  3.25it/s]


Epoch: 1, Loss: 0.6315, Acc: 0.6251, Lr: 0.00010


Epoch: 2, Loss: 0.6852, Acc: 0.0387: 100%|██████████| 1585/1585 [07:31<00:00,  3.51it/s]


...

Epoch: 12, Loss: 0.6302, Acc: 0.6264, Lr: 0.00010


Epoch: 13, Loss: 0.6463, Acc: 0.0405: 100%|██████████| 1585/1585 [06:03<00:00,  4.37it/s]


Epoch: 13, Loss: 0.6301, Acc: 0.6266, Lr: 0.00010


Epoch: 14, Loss: 0.6314, Acc: 0.6270:  44%|████▍     | 698/1585 [03:00<03:48,  3.88it/s]


KeyboardInterrupt: 

## 2.4 存储网络

In [69]:
torch.save(net.state_dict(), PKL_DIR_OUT)

## 3.1 输出 val 结果

In [19]:
from tqdm import tqdm

net = MyNet().to(DEVICE)
net.load_state_dict(torch.load(PKL_DIR_READ))
net.eval()

running_correct = 0

with tqdm(total=val_len) as pbar:
    for step, (bx, by) in enumerate(data_loader['val']):
        # 训练
        bx = bx.float().to(DEVICE)
        by = by.long().to(DEVICE)

        prediction = net(bx)

        # 统计成效
        pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
        my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
        my_acc = my_correct / BATCH_SIZE
        running_correct += my_correct

        # 更新进度条
        pbar.update(1)
        pbar.set_description('Correct: %d/%d, Acc: %.4f' % (
                              my_correct, BATCH_SIZE, my_correct / BATCH_SIZE))

# 输出一轮结果
print('Correct: %d/%d, Acc: %.4f' % (
       running_correct, val_size, running_correct / val_size))

Correct: 92/8192, Acc: 0.0112: 100%|██████████████████████████████████████████| 397/397 [01:27<00:00,  4.53it/s]


Correct: 2055658/3244167, Acc: 0.6336


## 4.1 导出 test 结果

In [18]:
result = data[np.isnan(data['是否回答'])]
result.iloc[-1]

问题id          Q2590286630
用户id          M3168697168
邀请创建时间           D3873-H9
性别                      1
是否回答                  NaN
用户二分类特征a           0.7121
用户二分类特征b        -0.652324
用户二分类特征c        -0.201699
用户二分类特征d        -0.591583
用户二分类特征e        -0.283857
用户多分类特征a             1092
用户多分类特征b              115
用户多分类特征c                6
用户多分类特征d              980
用户多分类特征e         0.343009
盐值                1.73947
访问评率            -0.995415
邀请问题间隔-day      -0.173304
关注话题命中数         -0.438016
Name: 1141682, dtype: object

In [30]:
result.loc[:, '是否回答'] = np.nan
result['是否回答']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6         NaN
7         NaN
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13        NaN
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
19        NaN
20        NaN
21        NaN
22        NaN
23        NaN
24        NaN
25        NaN
26        NaN
27        NaN
28        NaN
29        NaN
           ..
1141653   NaN
1141654   NaN
1141655   NaN
1141656   NaN
1141657   NaN
1141658   NaN
1141659   NaN
1141660   NaN
1141661   NaN
1141662   NaN
1141663   NaN
1141664   NaN
1141665   NaN
1141666   NaN
1141667   NaN
1141668   NaN
1141669   NaN
1141670   NaN
1141671   NaN
1141672   NaN
1141673   NaN
1141674   NaN
1141675   NaN
1141676   NaN
1141677   NaN
1141678   NaN
1141679   NaN
1141680   NaN
1141681   NaN
1141682   NaN
Name: 是否回答, Length: 1141683, dtype: float64

In [33]:
from tqdm import tqdm

net = MyNet().to(DEVICE)
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PKL_DIR_READ))
else:
    net.load_state_dict(torch.load(PKL_DIR_READ, map_location='cpu'))
net.eval()

i = 0
with tqdm(total=test_size) as pbar:
    for step, (bx, by) in enumerate(data_loader['test']):
        bx = bx.float().to(DEVICE)
        by = by.long().to(DEVICE)
        
        prediction = net(bx)
        
         # 统计成效
        pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
        print(pre)
        break
        my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
        my_acc = my_correct / BATCH_SIZE
        running_correct += my_correct
        
        pbar.update(1)
        i += 1

  0%|          | 0/1141683 [00:00<?, ?it/s]


TypeError: batch must contain tensors, numbers, dicts or lists; found object